In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
import importlib
import MIDItoSpectrogram
import random
# importlib.reload(MIDItoSpectrogram)
import mido
from sklearn.utils import shuffle
warnings.simplefilter(action = 'ignore', category = FutureWarning)
%matplotlib inline

Input Spectrogram Data

In [45]:
train_samples = []
train_labels = []

keys = ["CfMaj", "GfMaj", "DfMaj", "AfMaj", "EfMaj", "BfMaj", "FMaj", "CMaj", "GMaj", "DMaj", "AMaj", "EMaj", "BMaj", "FsMaj",  "CsMaj",
"AfMin", "EfMin", "BfMin", "FMin", "CMin", "GMin", "DMin", "AMin", "EMin", "BMin", "FsMin", "CsMin", "GsMin", "DsMin", "AsMin"]

importlib.reload(MIDItoSpectrogram)
chunks = 4000
for filename in os.listdir():
    if filename.endswith(".mid"):
        LABEL = []
        for i in range(len(keys)):
            try:
                filename.index(keys[i])
            except ValueError:
                continue
            else:
                temp = [0]*30
                temp[i] = 1

                LABEL = temp
                break
        
        mid = mido.MidiFile(filename, clip=True)
        # print(filename)
        result_array = MIDItoSpectrogram.mid2arry(mid)
        # print("len 1:", len(result_array), " Len 2: ", len(result_array[0]))
        result_array = result_array[len(result_array)%chunks : len(result_array)]
        result_array = np.transpose(result_array)
        # print("len 1:", len(result_array), " Len 2: ", len(result_array[0]))
        result_array = np.hsplit(result_array, len(result_array[0])/chunks) 
        print("len 1:", len(result_array), " Len 2: ", len(result_array[0]), " Len 3: ", len(result_array[0][0]))
        for i in range(len(result_array)):
            train_samples.append(result_array[i])
            train_labels.append(LABEL)

print("# of inputs ", len(train_samples))
train_samples = np.array(train_samples)
train_labels = np.array(train_labels)
train_labels, train_samples = shuffle(train_labels, train_samples)

        

len 1: 7  Len 2:  88  Len 3:  4000
len 1: 7  Len 2:  88  Len 3:  4000
len 1: 13  Len 2:  88  Len 3:  4000
len 1: 15  Len 2:  88  Len 3:  4000
len 1: 14  Len 2:  88  Len 3:  4000
len 1: 6  Len 2:  88  Len 3:  4000
len 1: 10  Len 2:  88  Len 3:  4000
len 1: 2  Len 2:  88  Len 3:  4000
len 1: 4  Len 2:  88  Len 3:  4000
len 1: 3  Len 2:  88  Len 3:  4000
len 1: 4  Len 2:  88  Len 3:  4000
len 1: 3  Len 2:  88  Len 3:  4000
len 1: 2  Len 2:  88  Len 3:  4000
len 1: 3  Len 2:  88  Len 3:  4000
len 1: 4  Len 2:  88  Len 3:  4000
len 1: 2  Len 2:  88  Len 3:  4000
len 1: 2  Len 2:  88  Len 3:  4000
len 1: 3  Len 2:  88  Len 3:  4000
len 1: 6  Len 2:  88  Len 3:  4000
len 1: 1  Len 2:  88  Len 3:  4000
len 1: 1  Len 2:  88  Len 3:  4000
len 1: 2  Len 2:  88  Len 3:  4000
len 1: 1  Len 2:  88  Len 3:  4000
len 1: 1  Len 2:  88  Len 3:  4000
len 1: 3  Len 2:  88  Len 3:  4000
len 1: 3  Len 2:  88  Len 3:  4000
len 1: 17  Len 2:  88  Len 3:  4000
len 1: 1  Len 2:  88  Len 3:  4000
# of inputs  14

Build Model

In [46]:
model = Sequential([
    Conv2D(filters = 32, kernel_size = (5,5), activation = 'relu', padding = 'same', input_shape = (88, 4000, 1)),
    MaxPool2D(pool_size = (5,5), strides = 5),
    Conv2D(filters = 32, kernel_size = (5, 5), activation = 'relu', padding = 'same'),
    MaxPool2D(pool_size = (5, 5), strides = 5),
    Flatten(),
    
    Dense(units = 30, activation = 'softmax')

])
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 88, 4000, 32)      832       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 17, 800, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 17, 800, 32)       25632     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 3, 160, 32)       0         
 g2D)                                                            
                                                                 
 flatten_12 (Flatten)        (None, 15360)             0         
                                                                 
 dense_12 (Dense)            (None, 30)              

Setup Internal Parameters

In [47]:
model.compile(optimizer = Adam(learning_rate = 0.005), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [48]:
model.fit(x = train_samples, y = train_labels,  validation_split = 0.2, batch_size = 32, epochs = 50, shuffle = True, verbose = 2)

Epoch 1/50
4/4 - 17s - loss: 73.0365 - accuracy: 0.1161 - val_loss: 3.1465 - val_accuracy: 0.2500 - 17s/epoch - 4s/step
Epoch 2/50
4/4 - 17s - loss: 3.1875 - accuracy: 0.1786 - val_loss: 3.3352 - val_accuracy: 0.0714 - 17s/epoch - 4s/step
Epoch 3/50
4/4 - 20s - loss: 3.1443 - accuracy: 0.2143 - val_loss: 4.0355 - val_accuracy: 0.3214 - 20s/epoch - 5s/step
Epoch 4/50
4/4 - 17s - loss: 2.9297 - accuracy: 0.3304 - val_loss: 3.1300 - val_accuracy: 0.1429 - 17s/epoch - 4s/step
Epoch 5/50
4/4 - 16s - loss: 2.1882 - accuracy: 0.4375 - val_loss: 4.9094 - val_accuracy: 0.1071 - 16s/epoch - 4s/step
Epoch 6/50
4/4 - 17s - loss: 1.6992 - accuracy: 0.5357 - val_loss: 5.7820 - val_accuracy: 0.2143 - 17s/epoch - 4s/step
Epoch 7/50
4/4 - 16s - loss: 1.0602 - accuracy: 0.7321 - val_loss: 6.6766 - val_accuracy: 0.1429 - 16s/epoch - 4s/step
Epoch 8/50
4/4 - 16s - loss: 0.7184 - accuracy: 0.8125 - val_loss: 10.1083 - val_accuracy: 0.1429 - 16s/epoch - 4s/step
Epoch 9/50
4/4 - 16s - loss: 0.5102 - accuracy

KeyboardInterrupt: 